In [21]:
#--Librerias--#
import pandas as pd 
from sklearn.model_selection import RandomizedSearchCV # búsqueda aleatoria de hiperparámetros 
import joblib  ### para guardar y cargar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.tree import DecisionTreeClassifier # Arboles de decision 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler ## escalar variable
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier #Modelo de clasificacion
from sklearn.metrics import confusion_matrix #### Matriz de confusion 
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns #####Graficos
from sklearn.metrics import accuracy_score #### medir precision del modelo 
import funciones as funciones

In [2]:
###----Lectura de datos----###
df = pd.read_csv("https://raw.githubusercontent.com/andressj1/Predicci-n-precio-de-m-viles-/refs/heads/main/Datos/df_fin", sep=",")

# Dummies

In [3]:
# Conversion; categoricas a dummies
df = pd.get_dummies(df, dtype = int)

In [16]:
# Separamos variables explicativas - variable objetivo
X = df.drop('price_range',axis=1)
y = df['price_range']

print(X.shape)
print(y.shape)

(1958, 20)
(1958,)


# Selección de variables

SelectKBest

*Anova*

In [19]:
def select_kbest(X,y,score_f,k):
    sel_kb = SelectKBest(score_func = score_f,k = k)
    sel_kb.fit(X,y)
    new_cols= sel_kb.get_support()
    print('Score:\n', sel_kb.scores_, '\n P-values:\n', sel_kb.pvalues_)
    return new_cols

In [20]:
X_new = select_kbest(X, y, f_classif, 15)
# Nuevo conjunto de datos
df_anova = X.iloc[:,X_new]
df_anova.head()

Score:
 [3.04920741e+01 3.35331284e-01 4.73780055e-01 8.91551611e-01
 1.22199528e+00 2.65059108e+00 1.50954726e+00 2.36569050e+00
 2.14732848e+01 1.47207299e+00 1.24648562e+00 3.02326974e-01
 1.54145222e+00 4.46116343e-01 2.77073594e+00 1.46647684e+03
 3.42825322e+02 1.26160727e+03 2.76292181e+00 2.61039561e+00] 
 P-values:
 [2.91490524e-019 7.99802009e-001 7.00575996e-001 4.44747836e-001
 3.00169037e-001 4.73176669e-002 2.10110111e-001 6.92057977e-002
 1.10460631e-013 2.20241669e-001 2.91318804e-001 8.23737633e-001
 2.01826723e-001 7.20074873e-001 4.02617550e-002 0.00000000e+000
 7.73411198e-179 0.00000000e+000 4.06876088e-002 4.99371431e-002]


,battery_power,four_g,int_memory,m_dep,n_cores,px_width,talk_time,touch_screen,tamaño_grande,tamaño_pequeño,ram_cat_2GB,ram_cat_3GB,ram_cat_4GB,mobile_wt_cat_liviano,mobile_wt_cat_pesado
0,842,0,7,0.6,2,756,19,0,0,1,0,1,0,0,1
1,1021,1,53,0.7,3,1988,7,1,1,0,0,1,0,1,0
2,563,1,41,0.9,5,1716,9,1,0,0,0,1,0,0,1
3,615,0,10,0.8,6,1786,11,0,1,0,0,1,0,1,0
4,1821,1,44,0.6,2,1212,15,1,0,1,1,0,0,0,1


RFE

In [22]:
# Función recursiva de selección de características
def recursive_feature_selection(X, y, model, k):
  rfe = RFE(model, n_features_to_select = k, step = 1)
  fit = rfe.fit(X, y)
  X_new = fit.support_
  print("Num Features: %s" % (fit.n_features_))
  print("Selected Features: %s" % (fit.support_))
  print("Feature Ranking: %s" % (fit.ranking_))

  return X_new

*Regresión Logistica*

In [23]:
model= LogisticRegression(max_iter = 1000,random_state = 42)
X_new=recursive_feature_selection(X, y, model, 15)
df_log = X.iloc[:,X_new]
df_log.info()

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Num Features: 15
Selected Features: [False  True  True False  True False  True  True False False  True  True
  True  True  True  True  True  True  True  True]
Feature Ranking: [5 1 1 3 1 4 1 1 6 2 1 1 1 1 1 1 1 1 1 1]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   blue                   1958 non-null   int64  
 1   dual_sim               1958 non-null   int64  
 2   four_g                 1958 non-null   int64  
 3   m_dep                  1958 non-null   float64
 4   n_cores                1958 non-null   int64  
 5   touch_screen           1958 non-null   int64  
 6   wifi                   1958 non-null   int64  
 7   tamaño_grande          1958 non-null   int32  
 8   tamaño_normal          1958 non-null   int32  
 9   tamaño_pequeño         1958 non-null   int32  
 10  ram_cat_2GB            1958 non-null   int32  

*Random Forest*

In [24]:
model = RandomForestClassifier(n_estimators=300,random_state=42,n_jobs=-1)
X_new = recursive_feature_selection(X,y,model,15)
df_ranfor = X.iloc[:,X_new]
df_ranfor.info()

Num Features: 15
Selected Features: [ True  True False  True  True  True  True  True  True  True  True  True
 False False False  True  True  True False  True]
Feature Ranking: [1 1 3 1 1 1 1 1 1 1 1 1 6 4 2 1 1 1 5 1]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1958 entries, 0 to 1957
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   battery_power         1958 non-null   int64  
 1   blue                  1958 non-null   int64  
 2   fc                    1958 non-null   int64  
 3   four_g                1958 non-null   int64  
 4   int_memory            1958 non-null   int64  
 5   m_dep                 1958 non-null   float64
 6   n_cores               1958 non-null   int64  
 7   px_width              1958 non-null   int64  
 8   talk_time             1958 non-null   int64  
 9   touch_screen          1958 non-null   int64  
 10  wifi                  1958 non-null   int64  
 11  ram_cat